In [1]:
%load_ext autoreload
%autoreload 1
%aimport my, data, net

import sys
import numpy as np
import pandas as pd

import os
import gc
import matplotlib.pyplot as plt
import importlib
import pickle

import my, data, net
from my import p
from tqdm.notebook import tqdm

pd.set_option('display.max_rows', 200)
pd.set_option("max_colwidth", 45)
pd.set_option("display.precision", 1)
pd.options.display.float_format = "{:.3f}".format
# pd.set_option("display.max_rows", 5)
# pd.reset_option("display.max_rows")

from sklearn.model_selection import train_test_split

# from pandarallel import pandarallel
# pandarallel.initialize(progress_bar=True)
     
dir_out = "out/"
dir_data = 'data/'

SEED = 34
np.random.seed(SEED)
N_CPU = os.cpu_count()

In [2]:
import torch
import pytorch_lightning as pl

In [3]:
target_info = pd.read_parquet('out/info_targets.pq')
target_info[:3]

,target_age,target_is_male,user_id,old_age,user_len,tel_price,user_len_bin,user_bin,fold,len_bucket
0,1,1,350459,31,178,5,5,50511,3,4
1,1,1,188276,35,111,7,5,50711,0,4
2,2,0,99002,41,639,6,7,70620,3,7


EMB DIM

In [4]:
encoders = my.load_pickle('out/encs.pik')
n_cats = {'price':12,'part_of_day':4, 'dayofweek': 7}
for name, e in encoders.items():
    n_cats[name] = len(e.classes_)
for k,v in n_cats.items():
    n_cats[k] = {'number':v,'emb_dim':my.emb_sz_rule(v)}
n_cats

/home/ubuntu/miniconda3/envs/mts_env/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


{'price': {'number': 12, 'emb_dim': 6},
 'part_of_day': {'number': 4, 'emb_dim': 3},
 'dayofweek': {'number': 7, 'emb_dim': 5},
 'region_name': {'number': 81, 'emb_dim': 19},
 'city_name': {'number': 985, 'emb_dim': 76},
 'cpe_manufacturer_name': {'number': 37, 'emb_dim': 12},
 'cpe_model_name': {'number': 599, 'emb_dim': 57},
 'url_host': {'number': 199683, 'emb_dim': 512},
 'cpe_type_cd': {'number': 4, 'emb_dim': 3},
 'cpe_model_os_type': {'number': 3, 'emb_dim': 3}}

In [5]:
all_plts = my.load_pickle('out/train_val_plts.pik')
len(all_plts)

270000

In [6]:
all_plts[350459]

{'region_name': tensor([72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72,
         72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72,
         72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72,
         72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72,
         72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72,
         72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72,
         72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72,
         72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72,
         72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72,
         72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72],
        dtype=torch.int32),
 'city_name': tensor([72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72,
         72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72,

In [6]:
target_info[:3]

,target_age,target_is_male,user_id,old_age,user_len,tel_price,user_len_bin,user_bin,fold,len_bucket
0,1,1,350459,31,178,5,5,50511,3,4
1,1,1,188276,35,111,7,5,50711,0,4
2,2,0,99002,41,639,6,7,70620,3,7


In [7]:
target_info['target_is_male'].value_counts()

 1    135332
 0    128994
-1      5674
Name: target_is_male, dtype: int64

In [8]:
is_male = target_info.loc[target_info['target_is_male']!=-1,['user_id','target_is_male']].set_index('user_id')['target_is_male'].to_dict()
is_male[188276]

1

In [15]:
for k in all_plts:
    all_plts[k]['y'] = float(is_male.get(k,-1))

TEST DATA

In [10]:
test_plts = my.load_pickle('out/test_plts.pik')
test_ids = test_plts['test_ids']
test_plts = test_plts['test_plts']
len(test_ids)

144724

In [11]:
for i, user_id in enumerate(test_ids):
    test_plts[i]['y'] = user_id

In [12]:
test_ds = my.SeqToTargetDataset(data = test_plts, target_col_name='y')

test_dl = torch.utils.data.DataLoader(
        dataset=test_ds,
        collate_fn=test_ds.collate_fn, 
        batch_size=128,
        shuffle=False,
        num_workers=1,
)

TRAIN

In [17]:
from functools import partial
from ptls.nn import TrxEncoder, RnnSeqEncoder, Head
from torchmetrics.classification import MulticlassF1Score

from pytorch_lightning.callbacks import EarlyStopping, TQDMProgressBar, ModelCheckpoint

feat_config ={
'time_depend_cats': {
    'url_host': {'in': 199683+1, 'out': 96},
    'region_name': {'in': 81+1, 'out': 16},
    'city_name': {'in': 985+1, 'out': 32},
    'dayofweek': {'in': 7+1, 'out': 5},
    'part_of_day': {'in': 4+1, 'out': 3},
    },
'time_depend_float': {
    'request_cnt':'identity',
    'diff_time':'identity'
    },
'const_cats': {
    'price': {'in': 12+1, 'emb_dim': 5},
    'cpe_manufacturer_name':{'in':37+1,'emb_dim':12},
    'cpe_model_name': {'in': 599+1, 'emb_dim': 24},
    'cpe_type_cd': {'in': 4+1, 'emb_dim': 3},
    'cpe_model_os_type': {'in': 3+1, 'emb_dim': 3}
}}

TARGET = 'is_male'
CHECKPOINT_DIR = 'ckpts/'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
CHECKPOINT_DIR, TARGET

('ckpts/', 'is_male')

In [ ]:
# !rm -rf ckpts

In [ ]:
from torchmetrics.classification import AUROC
preds = []

for fold in range(5):
    print('FOLD: ', fold)
    # net.set_seed(SEED*(fold+1))

    train_ids, val_ids = my.get_train_val_ids(target_info, fold=fold, target='target_is_male')
    print('LENS: ', len(train_ids), len(val_ids))

    val_plts = my.get_val_plts(all_plts, val_ids)

    train_ds = my.SeqToTargetDataset(data = all_plts, target_col_name='y', real_len=len(train_ids),seed=SEED*(fold+1),aug=True)
    val_ds = my.SeqToTargetDataset(data = val_plts, target_col_name='y', real_len=len(val_ids))

    train_sr = data.BucketBatchSampler(train_ids,batch_size=128, seed=SEED)

    train_dl = torch.utils.data.DataLoader(
        dataset=train_ds,
        collate_fn=train_ds.collate_fn, 
        batch_sampler = train_sr,
        num_workers=1,
    )

    val_dl = torch.utils.data.DataLoader(
        dataset=val_ds,
        collate_fn=val_ds.collate_fn, 
        batch_size=128,
        shuffle=False,
        num_workers=1,
    )

    seq_enc = RnnSeqEncoder(
    trx_encoder=TrxEncoder(
        embeddings= feat_config['time_depend_cats'],
        numeric_values=feat_config['time_depend_float'],
        embeddings_noise=0.001,
        # spatial_dropout=True,
        emb_dropout=0.1,
    ),
    hidden_size=256,
    # bidir=True,
    num_layers=2,
    dropout=0.2,
        trainable_starter=None
    )

    user_enc = net.UserEncoder(seq_enc=seq_enc, cat_embs=feat_config['const_cats'])

    pl_model = net.mySequenceToTarget(
        seq_encoder=user_enc, # user_enc.output_size
        head=Head(input_size=user_enc.output_size,
        hidden_layers_sizes=[10],drop_probs=[0.1], objective='classification', num_classes=1),
        loss=torch.nn.BCELoss(),
        metric_list=AUROC(task='binary', num_classes=2),
        optimizer_partial=partial(torch.optim.Adam, lr=0.002),
        lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=15, gamma=0.5),
    )

    tq = TQDMProgressBar(refresh_rate=10)

    es = EarlyStopping('val_BinaryAUROC', min_delta=0.001, patience=3,verbose=True, mode='max', check_on_train_epoch_end=False)

    chpt = ModelCheckpoint(dirpath=CHECKPOINT_DIR,filename=f'best_{TARGET}_f{fold}',  monitor='val_BinaryAUROC',mode='max')

    trainer = pl.Trainer(
        callbacks=[tq,es,chpt],
        max_epochs=9,
        # deterministic = True,
        gpus=1 if torch.cuda.is_available() else 0,
        gradient_clip_val = 10,
        # check_val_every_n_epoch=1
        val_check_interval = 0.99,
        enable_model_summary = True if fold==0 else False,
    )

    trainer.fit(pl_model, train_dl, val_dl)

    logit_preds = trainer.predict(dataloaders=test_dl, ckpt_path=chpt.best_model_path)

    fold_probs = np.concatenate(logit_preds)
    preds.append(fold_probs)
    np.save(f'out/preds_{TARGET}_f{fold}',fold_probs)

In [28]:
mean_fold_preds = np.mean(np.stack(preds), axis=0)
mean_fold_preds.shape

(144724,)

In [30]:
mean_fold_preds

array([0.49938482, 0.5176858 , 0.503586  , ..., 0.97375953, 0.8453118 ,
       0.96762484], dtype=float32)

In [32]:
sub = pd.DataFrame({'user_id':test_ids,TARGET:mean_fold_preds})
sub.to_csv(f'out/sub1_{TARGET}.csv',index=False)
sub

,user_id,is_male
0,1868,0.499
1,1989,0.518
2,2019,0.504
3,2022,0.595
4,2042,0.523
...,...,...
144719,412539,0.948
144720,413915,0.510
144721,414243,0.974
144722,414537,0.845


In [33]:
sub_age = pd.read_csv('out/sub1_age.csv')
sub_age

,user_id,age
0,1868,3
1,1989,3
2,2019,3
3,2022,3
4,2042,3
...,...,...
144719,412539,1
144720,413915,5
144721,414243,3
144722,414537,2


In [34]:
sub['age'] = sub_age['age']
sub

,user_id,is_male,age
0,1868,0.499,3
1,1989,0.518,3
2,2019,0.504,3
3,2022,0.595,3
4,2042,0.523,3
...,...,...,...
144719,412539,0.948,1
144720,413915,0.510,5
144721,414243,0.974,3
144722,414537,0.845,2


In [35]:
sub.to_csv('out/sub1.csv',index=False)